In [1]:
import torch
import time
from script.NeuralNets.Networks import SequentialNN
from script.settings import device, data_type
import script.DHOV.MultiDHOV as multidhov
from script.Verification.Verifier import SingleNeuronVerifier, MILPVerifier
import gurobipy as grp
from torchvision.datasets import MNIST
from torchvision.transforms import Compose, ToTensor, Normalize
from script.NeuralNets.ICNNFactory import ICNNFactory
import matplotlib.pyplot as plt
from script import eval

In [2]:
def add_output_constraints(model, nn_layer_out_bounds, label, output_vars, sovler_bound=1e-3):
    out_lb = nn_layer_out_bounds[-1][0]
    out_ub = nn_layer_out_bounds[-1][1]
    difference_lb = out_lb - out_ub[label]
    difference_ub = out_ub - out_lb[label]
    difference_lb = difference_lb.tolist()
    difference_ub = difference_ub.tolist()
    difference_lb.pop(label)
    difference_ub.pop(label)
    min_diff = min(difference_lb)
    max_diff = max(difference_ub)

    difference = model.addVars(9, lb=difference_lb, ub=difference_ub, name="diff_var")
    model.addConstrs((difference[i] == output_vars.tolist()[i] - output_vars.tolist()[label] for i in range(0, label)), name="diff_const0")
    model.addConstrs((difference[i - 1] == output_vars.tolist()[i] - output_vars.tolist()[label] for i in range(label + 1, 10)), name="diff_const1")

    max_var = model.addVar(lb=min_diff, ub=max_diff, name="max_var")
    model.addConstr(max_var == grp.max_(difference))

    if sovler_bound != None:
        model.setParam("BestObjStop", sovler_bound)

    model.update()
    model.setObjective(max_var, grp.GRB.MAXIMIZE)

In [3]:
def get_output_vars_snv_milp(model, output_size, output_layer_index):
    output_vars = []
    for i in range(output_size):
        output_vars.append(model.getVarByName("affine_var{}[{}]".format(output_layer_index, i)))
    output_vars = grp.MVar.fromlist(output_vars)
    return output_vars

In [4]:
def get_output_vars_dhov(model, output_size, output_layer_index):
    output_vars = []
    for i in range(output_size):
        output_vars.append(model.getVarByName("output_layer_[{}]_[{}]".format(output_layer_index, i)))
    output_vars = grp.MVar.fromlist(output_vars)
    return output_vars

In [5]:
def optimize_model(model, output_vars):
    start = time.time()
    model.update()
    model.optimize()
    if model.Status == grp.GRB.OPTIMAL or model.Status == grp.GRB.USER_OBJ_LIMIT:
        for i, var in enumerate(output_vars.tolist()):
            print("var {}: {}".format(i, var.getAttr("x")))
        max_var = model.getVarByName("max_var").getAttr("x")
        if max_var < 0:
            print("property verified with max difference {}".format(max_var))
            return True
        else:
             print("property NOT verified with max difference {}".format(max_var))
             return False

    elif model.Status == grp.GRB.INFEASIBLE:
        print("model infeasible")

        model.computeIIS()
        print("constraint")
        all_constr = model.getConstrs()

        for const in all_constr:
            if const.IISConstr:
                print("{}".format(const))

        print("lower bound")
        all_var = model.getVars()
        for var in all_var:
            if var.IISLB:
                print("{}, lb: {}, ub: {}".format(var, var.getAttr("lb"), var.getAttr("ub")))

        print("upper bound")
        all_var = model.getVars()
        for var in all_var:
            if var.IISUB:
                print("{}, lb: {}, ub: {}".format(var, var.getAttr("lb"), var.getAttr("ub")))

    print("time to optimize: {}".format(time.time() - start))
    return False

In [6]:
transform = Compose([ToTensor(),  Normalize(0.5, 0.5)])
training_data = MNIST(root="../../mnist", train=True, download=True, transform=transform)

nn = SequentialNN([28 * 28 * 1, 256, 256, 256, 256, 10])
nn.load_state_dict(torch.load("../../mnist_fc 4x256.pth", map_location=torch.device('cpu')), strict=False)
parameter_list = list(nn.parameters())
output_size = 10
number_layer = (len(parameter_list) - 2) // 2

eps = 0.026
samples_to_verify_count = 50

DHOV

SNR

In [15]:
snr_verifier_list = []
not_verified_index_snr = []
sample_index = 40
round_index = 0
while sample_index < samples_to_verify_count:
    image, label = training_data.__getitem__(sample_index)
    torch_image = torch.unsqueeze(image, 0).to(dtype=data_type).to(device)
    input_flattened = torch.flatten(torch_image)
    simple_bounds_affine_out, simple_bounds_layer_out = nn.calculate_box_bounds([input_flattened.add(-eps), input_flattened.add(eps)])

    if not torch.argmax(nn(torch_image)).item() == label:
        sample_index += 1
        continue
    print("{}/{} | image index {} ===========================================================================================".format(round_index, samples_to_verify_count-1, sample_index))

    overall_time = time.time()
    snv_verifier = SingleNeuronVerifier(nn, torch_image, eps, print_log=False, print_new_bounds=False, optimize_bounds=True)
    snv_verifier.generate_constraints_for_net()
    snr_verifier_list.append(snv_verifier)
    snv_model = snv_verifier.model.copy()
    snv_model.update()
    snv_out_vars = get_output_vars_snv_milp(snv_model, output_size, number_layer * 2)
    add_output_constraints(snv_model, simple_bounds_layer_out, label, snv_out_vars, sovler_bound=None)

    print("start with optimization")
    successful_ver = optimize_model(snv_model, snv_out_vars)
    if not successful_ver:
        not_verified_index_snr.append((sample_index, round_index))
    print("overall time: {}".format(time.time() - overall_time))

    round_index += 1
    sample_index += 1

0/49 | image index 40 ===========================================================================================
start with optimization
var 0: -7.418198921073439
var 1: 7.451668900662782
var 2: -0.3882476256098384
var 3: 1.789417360199626
var 4: -1.1221665558078042
var 5: -1.1631770121091334
var 6: -2.9622329985697826
var 7: -0.096712371024533
var 8: 5.63986897637917
var 9: 0.5211948392221872
property verified with max difference -1.811799924283612
overall time: 445.9088499546051
1/49 | image index 41 ===========================================================================================
start with optimization
var 0: -3.697307265499136
var 1: 1.88903549825306
var 2: 0.7894722372768914
var 3: 5.097959687334179
var 4: -4.870761160993947
var 5: 0.9083023134253763
var 6: -6.182785935165191
var 7: -3.238230998512689
var 8: 8.817070122347245
var 9: 2.1167444019455397
property verified with max difference -3.7191104350130653
overall time: 356.4559142589569
2/49 | image index 42 =======

In [7]:
print(not_verified_index_snr)

NameError: name 'not_verified_index_snr' is not defined

In [8]:
not_verified_index_snr = [(43, 3), (46, 6), (48, 8)]

In [10]:
group_size = 20
sample_count = 1000
sampling_method = "per_group_sampling"
net_size = [5, 1]
icnn_factory = ICNNFactory("logical", net_size, always_use_logical_layer=False)
#icnn_factory = ICNNFactory("standard", net_size, adapt_layer_for_init=True)
#icnn_factory = ICNNFactory("approx_max", net_size, maximum_function="SMU", function_parameter=0.3)

dhov_verifier_list = []
not_verified_index_dhov = []
sample_index = 0
round_index = 0
samples_to_verify_dhov = len(not_verified_index_snr)
for touple in not_verified_index_snr:
    sample_index = touple[0]
    round_index = touple[1]
    image, label = training_data.__getitem__(sample_index)
    torch_image = torch.unsqueeze(image, 0).to(dtype=data_type).to(device)
    input_flattened = torch.flatten(torch_image)
    #bounds_affine_out_snr, bounds_layer_out_snr = snr_verifier_list[round_index].bounds_affine_out, snr_verifier_list[round_index].bounds_layer_out
    bounds_affine_out_snr, bounds_layer_out_snr = None, None

    print("{}/{} | image index {} ===========================================================================================".format(round_index, samples_to_verify_dhov-1, sample_index))


    overall_time = time.time()
    dhov_verifier = multidhov.MultiDHOV()
    dhov_verifier.start_verification(nn, torch_image, icnn_factory, group_size, eps=eps, icnn_epochs=200, init_affine_bounds=bounds_affine_out_snr, init_layer_bounds=bounds_layer_out_snr,
                                     icnn_batch_size=10000, sample_count=sample_count, sample_new=True, use_over_approximation=True, break_after=None,
                                     sample_over_input_space=False, sample_over_output_space=True, tighten_bounds=True,
                                     use_fixed_neurons_in_grouping=False, sampling_method=sampling_method, layers_as_snr=[], layers_as_milp=[3],
                                     force_inclusion_steps=3, preemptive_stop=True, even_gradient_training=False,
                                     keep_ambient_space=False, data_grad_descent_steps=0, opt_steps_gd=100,
                                     train_outer=False, print_training_loss=False, print_new_bounds=False,
                                     grouping_method="consecutive", group_num_multiplier=5, store_samples=False, print_optimization_steps=False, print_last_loss=False,
                                     should_plot="none", optimizer="SdLBFGS", init_network=True, adapt_lambda="included")


    dhov_verifier_list.append(dhov_verifier)

    dhov_model = dhov_verifier.nn_encoding_model.copy()
    dhov_model.update()
    dhov_out_vars = get_output_vars_dhov(dhov_model, output_size, number_layer)
    add_output_constraints(dhov_model, dhov_verifier.bounds_layer_out, label, dhov_out_vars, sovler_bound=float("inf"))
    successful_ver = optimize_model(dhov_model, dhov_out_vars)
    if not successful_ver:
        not_verified_index_dhov.append(sample_index)
    print("overall time: {}".format(time.time() - overall_time))
    round_index += 1
    sample_index += 1

3/2 | image index 43 ===========================================================================================

approximation of layer: 0
    number of fixed neurons for current layer: 159
    layer progress, group 1 of 5 
        time for sampling for one group: 0.18610119819641113
        time for training: 12.06511116027832
        actual verification time 1.2580482959747314
        time for verification: 1.5271096229553223
    layer progress, group 2 of 5 
        time for sampling for one group: 0.11100006103515625
        time for training: 7.317829370498657
        actual verification time 1.3116767406463623
        time for verification: 1.5372240543365479
    layer progress, group 3 of 5 
        time for sampling for one group: 0.09700131416320801
        time for training: 5.6636528968811035
        actual verification time 4.196146726608276
        time for verification: 4.428155899047852
    layer progress, group 4 of 5 
        time for sampling for one group: 0.0839943